In [ ]:
struct outputscheduler
    opt::outputtimer
    opf::outputformater
end

function feed(outp::outputscheduler, xi, time, bounce)
    
    if add_output(outp.opf, xi, time, bounce)
        if outp.opf.tcounter > size(outp.opf.bt_skeleton)
            outp.opf.xi_skeleton = extend_skeleton_points(outp.opf.xi_skeleton, outp.opfsize_increment)
            outp.opf.bt_skeleton = extend_skeleton_points(outp.opf.bt_skeleton, outp.opfsize_increment)
        end
        outp.opf.xi_skeleton[:,opf.tcounter] = compress_xi(outp.opf, xi)
        outp.opf.bt_skeleton[:,opf.tcounter] = time
        outp.opf.tcounter +=1 
    end
    outp.opt = eval_stopping(outp.opt, xi, time, bounce)
    return outp
end

#-------------------------------------------------------------
abstract type outputtimer end

function is_running(opt::outputtimer)
    return opt.running
end

#-------------------------------------------------------------

abstract type outputformater end

function add_output(opf::outputformater, xi,time,bounce)
   return bounce 
end

function compress_xi(opf::outputformater, xi)
   return xi 
end

function extend_skeleton_points(skeleton_points, extension=100)
    m, n = size(skeleton_points)
    skeleton_new = zeros(m, n+extension)
    skeleton_new[:,1:n] = skeleton_points
    return skeleton_new
end 

#-------------------------------------------------------------

struct projopf <: outputformater
    d::Int64
    xi_skeleton::Array{Float64}
    bt_skeleton::Array{Float64}
    tcounter::Int64
    size_increment::Int64
    A::Array{Float64}
    d_out::Int64
end
projop(A, size_increment) = projopf(built_projopf(A, size_increment)...)

function built_projopf(A, size_increment)
    d_out, d = size(A)
    xi_skeleton = zeros(d,10*size_increment)
    bt_skeleton = zeros(1,10*size_increment)
    tcounter = 1
    return d, xi_skeleton, bt_skeleton, tcounter, size_increment, A, d_out
end

function compress_xi(outp::projops, xi)
   return outp.A @ xi  
end

#-------------------------------------------------------------

struct maxa_opt <: outputtimer
    running::Bool
    max_attempts::Int64
    acounter::Int64
end
maxa_opt(max_attempts) = maxa_opt(true, max_attempts, 1)

function eval_stopping(opt::maxa_opt, xi, time, bounce)
    acounter+=1
    if opf.acounter >= opf.max_attempts
        opf.running = false
    end
    return opt
end


In [ ]:
abstract type ll_model end
abstract type prior_model end

struct model
    ll::ll_model
    pr::prior_model
    bb::bound
end

function derivative_full(m::model,ξ, k)  
    d, Nobs = size(X)
    ξ = reshape(ξ,1,d)
    return sum([derivative(X[:,j], y[j], k, ξ, Nobs, σ) for j in 1:Nobs])[1]
end

In [ ]:
# ------------------------------------------------------------------
#                     BOUNDS
# ------------------------------------------------------------------

abstract type bound end

struct const_bound<:bound
    a::Float64
end

struct linear_bound<:bound
    a_fixed::Array{Float64}
    b_fixed::Array{Float64}
    a_xi::Array{Float64}
    b_xi::Array{Float64}
end
linear_bound(ll::ll_logistic, pr::gaussian_prior, gs_list::sampler_list) = 
linear_bound(build_linear_bound(ll, pr, gw)..., zeros(gs_list.d), zeros(gs_list.d))   


#------------------------- Bounds, no control variates -----------------------------# 

function build_linear_bound(ll::ll_logistic, pr::gaussian_prior, gs::mbsampler_list)
    d, Nobs = size(ll.X)
    σ2 = get_σ2(pr)
    a_fixed = [maximum(abs.(ll.X[i,:].*get_ubf(1:Nobs,gs.mbs[i]))) for i in 1:d]
    b_fixed = ones(d)./σ2
    return a_fixed, b_fixed
end

function update_bound(bb::linear_bound, ll::ll_logistic, pr::gaussian_prior, gs::mbsampler_list, ξ0, θ)
    bb.a_xi = abs.(ξ0-get_μ(pr))./get_σ2(pr)
    bb.b_xi = zeros(d)
    return bb
end

#------------------------- Bounds with control variates -----------------------------# 

function build_linear_bound(ll::ll_logistic, pr::gaussian_prior, gs::cvmbsampler_list)
    
    d, Nobs = size(ll.X)
    C_lipschitz = zeros(d, Nobs)
    for j in 1:Nobs
        C_lipschitz[:,j] = 1/4*[abs.(ll.X[i,j])*norm(ll.X[:,j]) for i in 1:d]
    end
    σ2 = get_σ2(pr)
    C = [maximum(C_lipschitz[i,:].*get_ubf(1:Nobs,gs.mbs[i])) for i in 1:d]
    a_fixed = zeros(d)
    b_fixed = √d*(C + 1/σ2)
    
    return a_fixed, b_fixed
end

function update_bound(bb::linear_bound, ll::ll_logistic, pr::gaussian_prior, gs::cvmbsampler_list, ξ0, θ)
    d = length(ξ0)
    norm_ = norm(gs.root-ξ0)
    bb.a_xi = [pos(θ[i]*gs.gradient_root_mean[i]) + norm_*bb.b_fixed[i]/√d for i in 1:d]
    bb.b_xi = zeros(d)
    return bb
end


function evaluate_bound(bb::linear_bound,t,k)
    return bb.a_fixed[k] + bb.a_xi[k] + t * (bb.b_fixed[k] + bb.b_xi[k] )  
end

In [19]:
function generate_event_time(bb::const_bound)
    
end

function generate_event_time(bb::linear_bound)
    
end
struct ll_logistic<:ll_model
    X::Array{Float64}
    y::Array{Int64}
end

function derivative(ll::ll_logistic, k, ξ, mb) 
    return [ll.X[k,j]*(exp.(ξ'll.X[:,j])./(1+exp.(ξ'll.X[:,j])) - ll.y[j]) for j in mb]
end

function estimate_ll_partial(ll::ll_logistic, k, ξ, mb, gs::mbsampler)
    return sum(derivative(ll::ll_logistic, k, ξ, mb).*get_ubf(gs,mb))
end

function estimate_ll_partial(ll::ll_logistic, k, ξ, mb, gs::cvmbsampler)
    return gs.gradient_root_mean[k] +  mean( (derivative(ll::ll_logistic, k, ξ, mb) 
            - gs.gradient_root[k,mb]).*get_ubf(gs,mb))
end

function derivative(ll::ll_logistic, k, ξ, mb) 
    d, Nobs = size(ll.X)
    rate_temp = mean([X[i0,j]*(exp.(ξ'X[:,j])./(1+exp.(ξ'X[:,j])) - y[j])/weights[j] for j in mb])
    return pos((θ[i0]*rate_temp)[1])
end

# Define abstract type for gaussian prior, sub-types of this abstract types must have attributes mu and sigma2  

function derivative(pp::prior_model, ξ) 
    d, = size(ξ)
    return derivative(pp, ξ, 1:d) 
end

abstract type gaussian_prior <:prior_model end
abstract type laplace_prior <:prior_model end

function get_σ2(pp::gaussian_prior)
    return pp.σ2
end

function get_μ(pp::gaussian_prior)
    return pp.μ
end

function derivative(pp::gaussian_prior, ξ, k) 
    return (ξ[k] - get_μ(pp)[k]) ./ (get_σ2(pp)[k])
end

LoadError: [91msyntax: expected "end" in definition of function "get_"[39m

In [ ]:
function get_event_time(ai, bi)     # for linear bounds
    # this assumed that bi is non-negative
    if bi > 0 
        u = rand()
        if ai >= 0 
            return (-ai + sqrt(ai^2 - 2*bi*log(u))) / bi
        else
            return -ai/bi + sqrt(-2*log(u)/bi)
        end
    elseif bi == 0
        return rand(Exponential(1/ai))
    else 
        print("Error, slope is negative \n")
    end
end

In [ ]:
function estimate_gradient(m::model,gw::mbsampler)
    mb = gsample(gw)
    
    return gradient_est, mb
end

function mbs_estimate(gw::mbsampler, f, x)
    mb = gsample(gw)
    return  sum(gw.ubf[mb].*map(f,(x[mb])))
end

In [ ]:
function ZZ_logistic(model::model, outp::outputscheduler, gs)

    d, Nobs = size(X) 

    ξ = copy(outp.opf.xi_skeleton[:,1])
    θ = ones(d)
    t = copy(outp.opf.bt_skeleton[1])
    
    bb = linear_bound(model.ll, model.pr, gs) 
    
#--------------------------------------------------------------------------------------------
    # run sampler:
    bounce = false
    while(is_running(outp.opt))
        
        bb = update_bound(bb, model.ll, model.pr, gs, ξ, θ)
        #-------------------------------------
        event_times = [get_event_time(bb.a_fixed[i] + bb.a_xi[i], bb.b_fixed[i] + bb.b_xi[i]) 
                       for i in 1:d]  
        τ, i0 = findmin(event_times)     
        #-------------------------------------
        t += τ 
        ξ_new = ξ + τ*θ
        
        rate_prior = pos(θ[i0]*derivative(model.pr, ξ_new, i0))
        
        mb = gsample(gs.mbs[i0])
        rate_likelihood = pos(θ[i0]*estimate_ll_partial(model.ll, i0, ξ_new, mb, gs.mbs[i0]))
        
        alpha = (rate_prior +rate_likelihood)/evaluate_bound(bb,τ,i0)
        if rand() < alpha
            θ[i0] *= -1
            switches += 1
            bounce = true
        else
            bounce = false
        end   
        outp = feed(outp, ξ_new, t, bounce)
    end
    return outp
end